In [5]:
from pandas_profiling import ProfileReport

from sklearn import metrics
from analyse_dataset import filling_rate, analyse, val_tri
from heatmap import heatmap, corrplot
from sklearn import decomposition
from sklearn import preprocessing
import math
import folium
import geopandas as gpd
import seaborn as sns
import os
import glob
import pandas as pd
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

In [45]:
data = pd.read_csv('application_train.csv',sep='\t',index_col=0)
data_final = pd.read_csv('data_final.csv',sep='\t')
data_prep = pd.read_csv('data_prep.csv',sep='\t')



In [6]:
with open('model_rf.pkl', 'rb') as model_file:
    model_rf = pickle.load(model_file)

In [7]:
data_final.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,FLAG_EMP_PHONE_0,FLAG_EMP_PHONE_1,FLAG_WORK_PHONE_0,FLAG_WORK_PHONE_1,FLAG_PHONE_0,FLAG_PHONE_1,FLAG_EMAIL_0,FLAG_EMAIL_1,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed or Students,...,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided,TARGET,vraie_valeur_y,proba_0,proba_1,predictions_y_cvp
0,347528,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.451582,-0.406909,-0.101403,-0.259043,-0.403595,-0.169809,0.984486,-0.694413,-0.281522,-0.354342,0.090752,-0.458926,-0.519393,-0.431152,-0.449196,-0.391356,-0.095794,-0.246054,-0.416355,-0.188154,0.983660,-0.693920,-0.286972,-0.363656,0.070545,-0.464753,-0.521844,-0.439991,-0.451516,-0.404534,-0.100054,-0.256250,-0.443799,1.114476,-0.324748,1.131367,-0.279141,-0.071837,-0.057811,-0.154684,-0.2699,-0.281582,0.315767,-0.758652,-0.586493,0.259879,0.932978,-0.094449,-0.465234,0.990317,1.063162,-1.320923,-0.333023,-0.456426,0.202334,1,1,0.005556,0.994444,1
1,63022,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,-0.451582,-0.560989,-0.101403,-0.259043,-0.588363,-0.521979,-1.012618,-0.694413,-0.281522,-0.354342,-0.704762,-0.736226,-0.519393,-0.431152,-0.449196,-0.551401,-0.095794,-0.246054,-0.596521,-0.531630,-1.013317,-0.693920,-0.286972,-0.363656,-0.716179,-0.738479,-0.521844,-0.439991,-0.451516,-0.559634,-0.100054,-0.256250,-0.567955,1.545817,

In [8]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import dash_bootstrap_components as dbc

In [11]:
def create_card(title, content,color):
    card = dbc.Card(
        dbc.CardBody(
            [
                html.H4(title, className="card-title"),
                html.Br(),
                html.H2(id=content, className="card-subtitle"),
                html.Br(),

                ]
        ),
        color=color, inverse=True
    )
    return(card)

In [58]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


gender_card = create_card('Genre du client', 'gender', 'info')
car_card = create_card('Possède une voiture ?', 'car', 'info')
realty_card = create_card('Possède une maison ?', 'realty', 'info')
childs_card = create_card('Nombre d enfants', 'childs', 'info')
fam_status_card = create_card('Situation familiale', 'fam_status', 'info')
age_card = create_card('Age du client', 'age', 'info')
contrat_card = create_card('Type de prêt :', 'contrat', 'info')
revenu_card = create_card('Revenu total :', 'revenu', 'info')
montant_credit_card = create_card('Montant emprunté :', 'montant_credit', 'info')
annuite_card = create_card('Montant annuité :', 'annuite', 'info')
prix_achat_card = create_card('Prix de l achat :', 'prix_achat', 'info')
proba_1_card = create_card('en % :', 'proba_1', 'info')
average_note_card = create_card('Note du client issu d organismes extérieurs (../1):', 'average_note', 'info')
age_car_card = create_card('Age de la voiture du client (années) :', 'age_car', 'info')
age_job_card = create_card('Ancienneté du client sur son poste (années) :', 'age_job', 'info')
recalcul_proba_1_card = create_card('Recalcul de la probabilité de défaut de paiement :', 'recalcul_proba_1', 'info')

app.layout = html.Div([
    html.I("Merci de rentrer l'ID client, svp."),
    html.Br(),
    dcc.Input(id='id_input', type='number',value=100002, style={'marginRight': '10px'}),
    html.Hr(),
    html.H3(" Informations générales :"),
    html.P(),
    dbc.Row([dbc.Col(id='gender_card', children=[gender_card], lg=4, width=4),
             dbc.Col(id='age_card', children=[age_card], lg=4, width=4),
             dbc.Col(id='fam_status_card', children=[fam_status_card], lg=4, width=4)]),
    html.P(),
    dbc.Row([dbc.Col(id='childs_card', children=[childs_card], lg=4, width=4),
             dbc.Col(id='car_card', children=[car_card], lg=4, width=4),
             dbc.Col(id='realty_card', children=[realty_card], lg=4, width=4)]),
    
    html.P(),
    dbc.Row([dbc.Col(id='age_car_card', children=[age_car_card], lg=4, width=4),
             dbc.Col(id='age_job_card', children=[age_job_card], lg=4, width=4)]),
    
    html.Hr(),
    html.H3("Informations financières"),
    html.P(),
    dbc.Row([dbc.Col(id='contrat_card', children=[contrat_card], lg=4, width=4),
             dbc.Col(id='revenu_card', children=[revenu_card], lg=4, width=4),
             dbc.Col(id='montant_credit_card', children=[montant_credit_card], lg=4, width=4)]),
    html.P(),
    dbc.Row([dbc.Col(id='annuite_card', children=[annuite_card], lg=4, width=4),
             dbc.Col(id='prix_achat_card', children=[prix_achat_card], lg=4, width=4),
             dbc.Col(id='average_note_card', children=[average_note_card], lg= 4, width = 4)]),
    html.Hr(),
    html.H3("Probabilité de défaut de paiement"),
    html.P(),
    dbc.Row([dbc.Col(id='proba_1_card', children=[proba_1_card], lg=4, width=6),
             dbc.Col(id='recalcul_proba_1_card', children=[recalcul_proba_1_card], lg=4, width=6)]),
    
    
    html.P(),
    html.P(),
    
    html.Div(id='slider-output-ext_source'),
    html.Br(),
    dcc.Slider(
        id='slider_ext_source',
        min=0,
        max=1,
        step=0.01,
        value=0.5),

    
    html.Div(id='slider-output-age_car'),
    html.Br(),
    dcc.Slider(
        id='slider_age_car',
        min=0,
        max=100,
        step=1,
        value=4),
    
    html.Div(id='slider-output-age_client'),
    html.Br(),
    dcc.Slider(
        id='slider_age_client',
        min=20,
        max=70,
        step=1,
        value=44),
    
    html.Div(id='slider-output-anciennete'),
    html.Br(),
    dcc.Slider(
        id='slider_anciennete',
        min=0,
        max=50,
        step=1,
        value=5),
    
    html.Div(id='slider-output-prix_bien'),
    html.Br(),
    dcc.Slider(
        id='slider_prix_bien',
        min=50000,
        max=4000000,
        step=25000,
        value=540000),
    
    html.Div(id='slider-output-montant_pret'),
    html.Br(),
    dcc.Slider(
        id='slider_montant_pret',
        min=50000,
        max=4000000,
        step=25000,
        value=540000),
    
    html.Div(id='slider-output-revenu'),
    html.Br(),
    dcc.Slider(
        id='slider_revenu',
        min=25000,
        max=4500000,
        step=25000,
        value=175000)
    
    
])

@app.callback(
    Output("gender", "children"),
    Output("car", "children"),
    Output("realty", "children"),
    Output("childs", "children"),
    Output("fam_status", "children"),
    Output("age", "children"),
    Output("contrat", "children"),
    Output("revenu", "children"),
    Output("montant_credit", "children"),
    Output("annuite", "children"),
    Output("prix_achat", "children"),
    Output("proba_1", "children"),
    Output("average_note","children"),
    Output("age_car","children"),
    Output("age_job","children"),
    Input("id_input", "value"),
)

def update_output_1(id_input):
    gender = data.loc[data.SK_ID_CURR == id_input].CODE_GENDER
    car = data.loc[data.SK_ID_CURR == id_input].FLAG_OWN_CAR
    realty = data.loc[data.SK_ID_CURR == id_input].FLAG_OWN_REALTY
    childs = data.loc[data.SK_ID_CURR == id_input].CNT_CHILDREN
    fam_status = data.loc[data.SK_ID_CURR == id_input].NAME_FAMILY_STATUS
    age = int(data.loc[data.SK_ID_CURR == id_input].DAYS_BIRTH_1)
    contrat = data.loc[data.SK_ID_CURR == id_input].NAME_CONTRACT_TYPE
    revenu = int(data.loc[data.SK_ID_CURR == id_input].AMT_INCOME_TOTAL)
    montant_credit = int(data.loc[data.SK_ID_CURR == id_input].AMT_CREDIT)
    annuite = int(data.loc[data.SK_ID_CURR == id_input].AMT_ANNUITY)
    prix_achat = float(data.loc[data.SK_ID_CURR == id_input].AMT_GOODS_PRICE)
    proba_1 = (data_final.loc[data_final.SK_ID_CURR == id_input].proba_1)*10000//1/100
    average_note = round(float(data.loc[data.SK_ID_CURR == id_input].EXT_SOURCE_AVG),2)
    age_car = int(data.loc[data.SK_ID_CURR == id_input].OWN_CAR_AGE)
    age_job = int(data.loc[data.SK_ID_CURR == id_input].DAYS_EMPLOYED_1)

    return gender, car, realty, childs, fam_status, age, contrat, revenu, montant_credit, annuite, prix_achat, proba_1,average_note,age_car, age_job
    

@app.callback(
    Output('slider_ext_source', 'value'),
    Output('slider_age_car','value'),
    Output('slider_age_client','value'),
    Output('slider_anciennete','value'),
    Output('slider_prix_bien','value'),
    Output('slider_montant_pret','value'),
    Output('slider_revenu','value'),
    Input("average_note", 'children'),
    Input("age_car", 'children'),
    Input("age", 'children'),
    Input("age_job", 'children'),
    Input("prix_achat", 'children'),
    Input("montant_credit", 'children'),
    Input("revenu", 'children'),
)

def update_slider(average_note, age_car, age, age_job, prix_achat, montant_credit, revenu):
    
    return average_note, age_car, age, age_job, prix_achat, montant_credit, revenu   




@app.callback(
    Output('slider-output-ext_source', 'children'),
    Output('slider-output-age_car','children'),
    Output('slider-output-age_client','children'),
    Output('slider-output-anciennete','children'),
    Output('slider-output-prix_bien','children'),
    Output('slider-output-montant_pret','children'),
    Output('slider-output-revenu','children'),
    Input('slider_ext_source', 'value'),
    Input('slider_age_car','value'),
    Input('slider_age_client','value'),
    Input('slider_anciennete','value'),
    Input('slider_prix_bien','value'),
    Input('slider_montant_pret','value'),
    Input('slider_revenu','value')
)


def update_output_2(value_source,car_age,client_age,anciennete,prix_bien,montant_pret,revenu):
    note_ext = 'Notation du client provenant des organismes extérieurs (../1): "{}"'.format(value_source)
    age_car = 'Nombre d annees du véhicule du client : "{}"'.format(car_age)
    age_client = 'Age du client : "{}"'.format(client_age)
    temps_travail = 'Années d ancienneté sur le poste actuel : "{}"'.format(anciennete)
    bien_prix = 'Prix du bien pour lequel est souscrit le prêt : "{}"'.format(prix_bien)
    pret_montant = 'Montant du prêt : "{}"'.format(montant_pret)
    revenu = 'Revenu annuel total du client :"{}"'.format(revenu)
    
    return note_ext,age_car,age_client,temps_travail,bien_prix,pret_montant,revenu

@app.callback(
    Output('recalcul_proba_1','children'),
    Input('slider_ext_source', 'value'),
    Input('slider_age_car','value'),
    Input('slider_age_client','value'),
    Input('slider_anciennete','value'),
    Input('slider_prix_bien','value'),
    Input('slider_montant_pret','value'),
    Input('slider_revenu','value'),
    Input("id_input", "value")
)

def recalcul_proba(value_source,car_age,client_age,anciennete,prix_bien,montant_pret,revenu,id_client):
    data_predict = data_prep.drop(columns=['TARGET'])
    data_predict = data_predict.set_index('SK_ID_CURR')
    infos = data_predict.loc[data_predict.index == id_client]
    infos.at[id_client,'EXT_SOURCE_AVG']= (value_source-0.504583)/0.15203
    infos.at[id_client,'OWN_CAR_AGE']= (car_age-4.119)/9.07
    infos.at[id_client,'DAYS_BIRTH_1']=(client_age-43.81847)/11.9546
    infos.at[id_client,'DAYS_EMPLOYED_1']=(anciennete-5.316719)/6.27111
    infos.at[id_client,'AMT_GOODS_PRICE']=(prix_bien - 536400.2)/367570.6
    infos.at[id_client,'AMT_CREDIT']=(montant_pret - 597477.6) / 400494
    infos.at[id_client,'AMT_INCOME_TOTAL']=(revenu - 168099.8)/98898
    
    return model_rf.predict_proba(infos)[0][1]
    
    

if __name__ == '__main__':
    app.run_server(debug=True, port=8052, mode='external')

Dash app running on http://127.0.0.1:8052/


In [51]:
data_predict = data_prep.drop(columns=['TARGET'])
data_predict = data_predict.set_index('SK_ID_CURR')
infos = data_predict.loc[data_predict.index == 100002]
infos

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,FLAG_EMP_PHONE_0,FLAG_EMP_PHONE_1,FLAG_WORK_PHONE_0,FLAG_WORK_PHONE_1,FLAG_PHONE_0,FLAG_PHONE_1,FLAG_EMAIL_0,FLAG_EMAIL_1,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed or Students,OCCUPATION_TYPE_Waiters/barmen staff,...,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,-0.366665,-0.046663,-0.192909,0.361808,0.167772,-0.162432,-0.360187,-0.101403,-0.259043,-0.322595,0.034113,0.968997,1.05115,0.038325,-0.354342,-0.025325,-0.18229,0.374055,0.190308,-0.150336,-0.343104,-0.095794,-0.246054,-0.337443,0.011516,0.968171,1.032284,0.024446,-0.363656,-0.04425,-0.191684,0.362054,0.170102,-0.161579,-0.358451,-0.100054,-0.25625,-0.407123,0.251794,4.160535,0.261273,5.245355,-0.071837,-0.057811,-0.154684,-0.2699,-0.281582,-0.442267,-1.49691,-0.56952,-0.374245,-0.570934,-0.094449,-1.150135,0.51071,-0.654184,0.789472,-2.254804,0.215303,0.202334


In [57]:
model_rf.predict_proba(infos)[0][1]

0.9055555555555556